# **Codigo del proyecto**:

Análisis de precios de las acciones de las 3 principales compañías de producción de GPUs en serie del mercado (Nvidia, AMD y Intel) en el periodo del 1980-2024.

**Samsung Innovation Campus:**

Venezuela. VT10.

Grupo 1:

Johnny Diaz.

Guillermo Cardenas.



In [15]:
import pandas as pd  # Importa la biblioteca pandas y le asigna el alias pd
import plotly.express as px  # Importa la biblioteca plotly.express y le asigna el alias px

# Carga los datos de AMD, INTEL y NVIDIA desde un archivo CSV y los almacena en un DataFrame
df_AMD = pd.read_csv('AMD.csv')
df_INTEL = pd.read_csv('INTEL.csv')
df_NVIDIA = pd.read_csv('NVIDIA.csv')

# Convierte la columna 'Date' de los DataFrame df_AMD, df_INTEL y df_NVIDIA al tipo de dato datetime para facilitar su manipulación
df_AMD['Date'] = pd.to_datetime(df_AMD['Date'])
df_INTEL['Date'] = pd.to_datetime(df_INTEL['Date'])
df_NVIDIA['Date'] = pd.to_datetime(df_NVIDIA['Date'])


In [16]:
# Calcula el promedio de las columnas 'Low' y 'High' para cada fila de los 3 DataFrame y lo almacena en una nueva columna 'Average'
df_INTEL["Average"] = df_INTEL[["Low", "High"]].mean(axis=1)
df_AMD["Average"] = df_AMD[["Low", "High"]].mean(axis=1)
df_NVIDIA["Average"] = df_NVIDIA[["Low", "High"]].mean(axis=1)


In [17]:
# Crea un nuevo DataFrame combinando columnas específicas de los tres DataFrames 
# y agrega una columna 'Empresa' con el nombre respectivo de cada empresa como valor
df_combinado = pd.concat([
    df_AMD[['Date', 'Average',"Volume"]].assign(Empresa='AMD'), 
    df_INTEL[['Date', 'Average',"Volume"]].assign(Empresa='INTEL'),
    df_NVIDIA[['Date', 'Average',"Volume"]].assign(Empresa='NVIDIA')  
])

# Crea la gráfica con Plotly Express
fig = px.line(df_combinado, x='Date', y='Average', color='Empresa', 
              title='Gráfico de Precios Promedios',
              color_discrete_map={'AMD': '#ff6961','INTEL': '#636EFA','NVIDIA': '#00cc96'})  # Define los colores específicos para cada empresa

fig.show()  # Muestra el gráfico


In [18]:
# Crea una gráfica de línea con Plotly Express usando el DataFrame combinado

fig = px.line(df_combinado, x='Date', y="Volume", color='Empresa',
              title='Gráfico de Volumen de Acciones',
              color_discrete_map={'AMD': '#ff6961','INTEL': '#636EFA','NVIDIA': '#00cc96'})  # Define los colores específicos para cada empresa

fig.show()  # Muestra el gráfico

In [19]:
# Establece la columna 'Date' como el índice de los DataFrame y los asigna a un nuevo DataFrame
new_df_AMD = df_AMD.set_index(["Date"])
new_df_INTEL = df_INTEL.set_index(["Date"])
new_df_NVIDIA = df_NVIDIA.set_index(["Date"])


In [20]:
# Combina los DataFrames de AMD, INTEL y NVIDIA en un solo DataFrame, df_all
df_all = pd.concat([new_df_AMD[["High", "Low", "Average"]], new_df_INTEL[["High", "Low", "Average"]], new_df_NVIDIA[["High", "Low", "Average"]]],
                   axis=1, keys=['AMD', 'INTEL', 'NVIDIA'])  # Asigna claves para diferenciar cada DataFrame combinado

# Rellena los valores NaN en df_all con ceros y asigna el resultado a df_all_new
df_all_new = df_all.fillna(0)
df_all_new.head()


AMD                         INTEL                     NVIDIA  \
                High       Low   Average      High       Low   Average   High   
Date                                                                            
1980-03-18  3.125000  2.937500  3.031250  0.328125  0.322917  0.325521    0.0   
1980-03-19  3.083333  3.020833  3.052083  0.335938  0.330729  0.333334    0.0   
1980-03-20  3.062500  3.010417  3.036459  0.334635  0.329427  0.332031    0.0   
1980-03-21  3.020833  2.906250  2.963541  0.322917  0.317708  0.320312    0.0   
1980-03-24  2.916667  2.635417  2.776042  0.316406  0.311198  0.313802    0.0   

                         
            Low Average  
Date                     
1980-03-18  0.0     0.0  
1980-03-19  0.0     0.0  
1980-03-20  0.0     0.0  
1980-03-21  0.0     0.0  
1980-03-24  0.0     0.0

In [21]:
# Obtiene los valores máximos de la columna 'High' para cada empresa y asigna los nombres de las empresas a los índices
df_max = df_all.xs('High', level=1, axis=1).max()
df_max.index = ['AMD', 'INTEL', 'NVIDIA']  # Asigna nombres de empresas a los índices del resultado

# Obtiene los valores mínimos de la columna 'Low' para cada empresa y asigna los nombres de las empresas a los índices
df_min = df_all.xs('Low', level=1, axis=1).min()
df_min.index = ['AMD', 'INTEL', 'NVIDIA']  # Asigna nombres de empresas a los índices del resultado

# Calcula el promedio de la columna 'Average' para cada empresa y asigna los nombres de las empresas a los índices
df_average = df_all.xs('Average', level=1, axis=1).mean()
df_average.index = ['AMD', 'INTEL', 'NVIDIA']  # Asigna nombres de empresas a los índices del resultado


In [22]:
# Combina los DataFrames df_min, df_average y df_max en un solo DataFrame, df_min_average_max, asignando claves a cada DataFrame
df_min_average_max = pd.concat([df_min, df_average, df_max],
                               axis=1, keys=["Min", "Average", "Max"])  # Asigna claves 'Min', 'Average' y 'Max' a cada DataFrame combinado

# Muestra las primeras 5 filas del DataFrame df_min_average_max
df_min_average_max.head()


,Min,Average,Max
AMD,1.610000,19.117979,227.300003
INTEL,0.216146,20.240026,75.828125
NVIDIA,0.333333,49.361517,974.000000


In [23]:
# Asigna un nombre a las columnas del DataFrame df_min_average_max
df_min_average_max.columns.name = 'Valores de las columnas'

# Crea un gráfico de histogramas con Plotly Express
fig = px.histogram(df_min_average_max,
                   x=df_min_average_max.index,  # Usa los índices (nombres de las empresas) en el eje x
                   y=df_min_average_max.columns,  # Usa las columnas (valores históricos: Min, Average, Max) en el eje y
                   barmode="group",  # Agrupa las barras para comparar los valores
                   title="Comparativa valores históricos", 
                   text_auto='.2s')  # Añade texto automático en las barras con formato

# Actualiza los títulos de los ejes del gráfico
fig.update_layout(xaxis_title='Empresas',  # Título del eje x
                  yaxis_title='Valor monetario ($)')  # Título del eje y

# Muestra el gráfico
fig.show()


In [24]:
# Combina las columnas 'High' de los DataFrames de AMD, INTEL y NVIDIA en un solo DataFrame
df_all_high = pd.concat([new_df_AMD["High"], new_df_INTEL["High"], new_df_NVIDIA["High"]], axis=1, keys=['AMD', 'INTEL', 'NVIDIA'])

# Rellena los valores NaN con ceros
df_all_high = df_all_high.fillna(0)
df_all_high.head()


,AMD,INTEL,NVIDIA
Date,,,
1980-03-18,3.125000,0.328125,0.0
1980-03-19,3.083333,0.335938,0.0
1980-03-20,3.062500,0.334635,0.0
1980-03-21,3.020833,0.322917,0.0
1980-03-24,2.916667,0.316406,0.0


In [25]:
# Encuentra el índice de los valores máximos en cada fila del DataFrame df_all_high y resetea el índice, eliminando la columna 'Date'
df_max = df_all_high.idxmax(axis=1).reset_index().drop("Date", axis=1)

# Calcula los porcentajes de frecuencia de los valores en df_max y los convierte a porcentajes
counts = df_max.value_counts(normalize=True) * 100
counts = counts.reset_index()

# Renombra las columnas del DataFrame counts
counts.columns = ["Empresa", "Porcentaje"]
counts


,Empresa,Porcentaje
0,INTEL,46.632451
1,AMD,40.473618
2,NVIDIA,12.893931


In [26]:
fig = px.pie(counts,  # Crea un gráfico de pastel usando el DataFrame 'counts'
             values='Porcentaje',  # Usa los valores de la columna 'Porcentaje' para los tamaños de los segmentos
             names='Empresa',  # Usa los valores de la columna 'Empresa' para los nombres de los segmentos
             title='Porcentaje de Categorías',  # Asigna un título al gráfico
             color='Empresa',  # Colorea los segmentos según la empresa
             color_discrete_map={'AMD': '#ff6961','INTEL': '#636EFA','NVIDIA': '#00cc96'}  # Asigna colores específicos a cada empresa
             )
fig.show()  # Muestra el gráfico


In [30]:
# Combina las columnas 'Volume' de los DataFrames de AMD, INTEL y NVIDIA en un solo DataFrame
df_volume = pd.concat([new_df_AMD["Volume"], new_df_INTEL["Volume"], new_df_NVIDIA["Volume"]],
                      axis=1, keys=['AMD', 'INTEL', 'NVIDIA'])  # Asigna claves para diferenciar cada serie combinada

# Suma los volúmenes para cada empresa y restablece el índice
volumes = df_volume.sum().reset_index()

# Renombra las columnas del DataFrame volumes
volumes.columns = ["Empresa", "Volumen"]
volumes


,Empresa,Volumen
0,AMD,2.137743e+11
1,INTEL,5.593724e+11
2,NVIDIA,3.857568e+11


In [28]:
# Crea un gráfico de barras usando el DataFrame 'volumes'
fig = px.bar(volumes,
             x="Empresa",
             y="Volumen",
             color="Empresa",
             title="Movimientos Históricos de las acciones",
             color_discrete_map={'AMD': '#ff6961', 'INTEL': '#636EFA', 'NVIDIA': '#00cc96'}  # Asigna colores específicos a cada empresa
             )

# Actualiza los títulos de los ejes del gráfico
fig.update_layout(xaxis_title='Empresas',
                  yaxis_title='Volumen Histórico ($)')

# Muestra el gráfico
fig.show()
